methodology

In [3]:
pip install scanpy anndata

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata

patient data

In [6]:
patient_d = pd.read_excel("NIHMS1531727-supplement-2.xlsx", header=3, nrows=15, keep_default_na=False)

In [34]:
BCC = patient_d["Tumor Type"] == "BCC"
pos_response = patient_d["Response"] == "Yes"
BCC_dataset = BCC_dataset[BCC & pos_response]
BCC_dataset # run when ready, future goals: select some more, organize data better

/tmp/ipykernel_449689/2727134200.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  BCC_dataset = BCC_dataset[BCC & pos_response]


,Patient,Tumor Type,Treatment,Ongoing Vismodegib treatment,Prior treatment,Response,Best % change,scRNA pre site,scRNA days pre treatment,scRNA post site,...,Adaptive pre site,Adaptive days pre treatment,Adaptive post site,Adaptive days post treatment,PBMC Adaptive days pre treatment,PBMC Adaptive days post treatment,Exome pre site,Exome days pre treatment,Exome post site,Exome days post treatment
0,su001,BCC,Pembrolizumab,+,Vismodegib,Yes,-81,L arm,"-239, -78",L arm,...,L arm,"-78, -5",L arm,"83, 146",-239,104,L arm,-78,L arm,146
1,su002,BCC,Pembrolizumab,-,Vismodegib,Yes,0*,Nose,0,Nose,...,Nose,-602,NA,NA,NA,117,Nose,0,Nose,62
2,su003,BCC,Pembrolizumab,-,Vismodegib,Yes,-100,R arm,-243,R chest,...,NA,NA,R chest,15,-243,16,R arm,-244,R arm,155
3,su004,BCC,Cemiplimab,-,-,Yes,-25,Knee,1,Knee,...,NA,NA,NA,NA,NA,NA,Knee,0,NA,NA
8,su009,BCC,Cemiplimab,-,Sonidegib,Yes,-62,L forehead,0,L forehead,...,L forehead,0,L forehead,21,NA,NA,NA,NA,NA,NA
12,su012,BCC,Cemiplimab,-,Vismodegib,Yes,-57,L cheek,0,L cheek,...,L cheek,0,L cheek,21,NA,NA,NA,NA,NA,NA


tcr data

In [66]:
tcr_d = pd.read_table("bcc_tcr.txt", header=0)
# tcr_d[0] = tcr_d[0].map(lambda x: x.lstrip("bcc.su001"))
# tcr_d["T-cell"] = tcr_d["T-cell"].str[-16:]

# grab TRB data that is in the TRA column by mistake
# tcr_d["wrong_TRA"] = tcr_d['TRA_raw'].where(tcr_d['TRA_raw'].str.startswith('TRB:'), None)
# remove TRB data from TRA column
# tcr_d['TRA_raw'] = tcr_d['TRA_raw'].where(~tcr_d['TRA_raw'].str.startswith('TRB:'), None)
# grab TRA data that is in the TRB column by mistake
# tcr_d["wrong_TRB"] = tcr_d['TRB_raw'].where(tcr_d['TRB_raw'].str.startswith('TRA:'), None)
# remove TRA data from TRB column
# tcr_d['TRB_raw'] = tcr_d['TRB_raw'].where(~tcr_d['TRB_raw'].str.startswith('TRA:'), None)
# create new TRA column that combines TRA_raw + wrong_TRB
# tcr_d['TRA'] = tcr_d['TRA_raw'].str.cat(tcr_d['wrong_TRB'], sep=' ')
# create new TRB column that combines wrong_TRB + TRB_raw
# tcr_d['TRB'] = tcr_d['wrong_TRA'].str.cat(tcr_d['TRB_raw'], sep=' ')
# quick check to see if it works -> tcr_d.iloc[0, 8]
# output: 'TRB:TGCGCCAGCCGCCTAGCGGGGGGGCTCCAAGAGACCCAGTACTTC TRB:CASRLAGGLQETQYF'. looks good!
# we no longer need any column besides 'TRA' and 'TRB'
# tcr_d = tcr_d.drop(columns=['TRB_raw', 'TRA_raw', 'wrong_TRB', 'wrong_TRA'])
# personal preference as i don't like NaN, will remove if it causes problems later
# tcr_d = tcr_d.replace(np.NaN, 0)
# tcr_d["TRA"] = tcr_d["TRA"].str[4:]
# tcr_d["TRB"] = tcr_d["TRB"].str[4:]
tcr_d

,cdr3s_nt,cdr3s_aa
bcc.su001.pre.tcell_AAACCTGCAGATCGGA,TRB:TGCGCCAGCCGCCTAGCGGGGGGGCTCCAAGAGACCCAGTACTTC,TRB:CASRLAGGLQETQYF
bcc.su001.pre.tcell_AAACGGGTCATAGCAC,TRA:TGTGCAGAGACTATTTTGTACAGCAGTGCTTCCAAGATAATC...,TRA:CAETILYSSASKIIF;TRB:CAWTTPGTSNSPLHF
bcc.su001.pre.tcell_AAAGATGAGACAGGCT,TRA:TGCATCGTCAGTCTTTCTCTTGTGATTTATAACCAGGGAGGA...,TRA:CIVSLSLVIYNQGGKLIF;TRB:CASSSSWEGSPGEQYF
bcc.su001.pre.tcell_AAAGATGCACAAGCCC,TRA:TGTGCTGTGGAGCGAAATACTGGAGGCTTCAAAACTATCTTT...,TRA:CAVERNTGGFKTIF;TRB:CASSQLGNGNQPQHF
bcc.su001.pre.tcell_AAAGATGTCTGAGTGT,TRA:TGTGCTGGGGGGAATTATGGTGGTGCTACAAACAAGCTCATC...,TRA:CAGGNYGGATNKLIF;TRB:CASSLPGARVAFF
...,...,...
bcc.su012.post.tcell_TTTGCGCGTCTTTCAT,TRB:TGTGCCAGCAGCTTGCGACAGGGAGCTGGCAGCAATCAGCCC...,TRB:CASSLRQGAGSNQPQHF
bcc.su012.post.tcell_TTTGGTTCAGCCAGAA,TRA:TGTGCAGCAAGTGAAAACTACAAGCTCAGCTTT;TRB:TGTG...,TRA:CAASENYKLSF;TRB:CASSGEGRYGYTF
bcc.su012.post.tcell_TTTGTCAAGCACCGTC,TRA:TGTGCTGTGAGTACTAATGACATGCGCTTT;TRB:TGTGCCA...,TRA:CAVSTNDMRF;TRB:CASSPIGSYEQYF
bcc.su012.post.tcell_TTTGTCAAGTGAACGC,TRB:TGTGCCAGCAGTTTCCTCACAGGGGTCAACGAGCAGTACTTC,TRB:CASSFLTGVNEQYF


scrna data

In [62]:
scrna_d = sc.read_csv("bcc_scRNA_counts.txt", delimiter="\t", first_column_names=True).transpose()
scrna_d.var_names_make_unique()

In [63]:
# Add QC metrics to the AnnData object
sc.pp.calculate_qc_metrics(scrna_d, inplace=True, log1p=True)

# Plot QC metrics to understand the data
# sc.pl.violin(scrna_d, ['n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes'], jitter=0.4)
# sc.pl.scatter(scrna_d, x='total_counts', y='log1p_n_genes_by_counts')

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
# scrna_d.var["pre"] = scrna_d.var_names.str.startswith("bcc.su001.pre")
# ribosomal genes
# scrna_d.var["ribo"] = scrna_d.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
# scrna_d.var["hb"] = scrna_d.var_names.str.contains("^HB[^(P)]")

scrna_d.obs

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes
bcc.su001.pre.tcell_AAACCTGCAGATCGGA,1110,7.013016,3061.0,8.026824,39.104868,54.459327,66.481542,80.071872
bcc.su001.pre.tcell_AAACCTGCAGGGATTG,620,6.431331,1217.0,7.104965,37.880033,52.095316,65.488907,90.139688
bcc.su001.pre.tcell_AAACGGGCATAGACTC,973,6.881411,1559.0,7.352441,28.608082,37.652341,50.416934,69.660038
bcc.su001.pre.tcell_AAACGGGTCATACGGT,1210,7.099202,2596.0,7.862112,38.636364,46.302003,55.739599,72.650231
bcc.su001.pre.tcell_AAACGGGTCATAGCAC,1529,7.333023,4174.0,8.336869,38.883565,51.964542,62.122664,75.347389
...,...,...,...,...,...,...,...,...
bcc.su012.post.tcell_TTTGCGCGTCTTTCAT,1467,7.291656,4601.0,8.434246,39.491415,54.031732,65.464030,78.982830
bcc.su012.post.tcell_TTTGGTTCAGCCAGAA,1356,7.213032,3101.0,8.039803,30.312802,42.115447,54.466301,72.396001
bcc.su012.post.tcell_TTTGTCAAGCACCGTC,1106,7.009409,2595.0,7.861727,35.645472,48.516378,60.385356,76.647399
bcc.su012.post.tcell_TTTGTCAAGTGAACGC,734,6.599870,1327.0,7.191429,33.609646,45.968350,59.758855,82.366240
